# Video Streaming using Icecast/P2PSP

* Cristóbal Medina López
* Juan Pablo García Ortiz
* Juan Alvaro Muñoz Naranjo
* José Juan Sánchez Hernández
* Leocadio González Casado
* Max Mertens
* Vicente González Ruiz

[SAL, UAL](http://www.hpca.ual.es/)

Dic 7, 2017

[https://github.com/P2PSP/slides](https://github.com/P2PSP/slides)

<img src="data/thanks.png" style="width: 600px;" align="middle"/>

## Internet transmission models

| MODE | TOPOLOGY | SCOPE | PROTOCOLS | APPLICATIONS/SYSTEMS |
|:----:|:--------:|:-----:|:---------:|:--------------------:|
| [Unicast](https://en.wikipedia.org/wiki/Unicast) | <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/7/75/Unicast.svg/200px-Unicast.svg.png" style="width: 200px;" align="middle"/> | Whole network | [HTTP](https://en.wikipedia.org/wiki/Hypertext_Transfer_Protocol) [(TCP)](https://en.wikipedia.org/wiki/Transmission_Control_Protocol) | [YouTube/Netflix](https://www.reddit.com/r/networking/comments/2cp356/how_do_streaming_services_like_netflix_and/) |
| [Broadcast](https://en.wikipedia.org/wiki/Broadcasting_%28networking%29) | <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/d/dc/Broadcast.svg/250px-Broadcast.svg.png" style="width: 200px;" align="middle"/> | Subnet (LAN) | [ARP](https://en.wikipedia.org/wiki/Address_Resolution_Protocol) | - |
| [Multicast](https://en.wikipedia.org/wiki/Multicast) | <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/30/Multicast.svg/250px-Multicast.svg.png" style="width: 200px;" align="middle"/> | Defined horizon alg. (routers/TTL) | [SLP](https://en.wikipedia.org/wiki/Service_Location_Protocol) [(UDP)](https://en.wikipedia.org/wiki/User_Datagram_Protocol), [SDP](https://en.wikipedia.org/wiki/Session_Description_Protocol) [(UDP)](https://en.wikipedia.org/wiki/User_Datagram_Protocol) | [Movistar+](http://www.movistar.es/), [Ono TV](https://www.ono.es/television/) |
| [Anycast](https://en.wikipedia.org/wiki/Anycast) | <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/4/43/Anycast.svg/250px-Anycast.svg.png" style="width: 200px;" align="middle"/> | Internet | [DNS protocol](https://en.wikipedia.org/wiki/Domain_Name_System) [(UDP)](https://en.wikipedia.org/wiki/User_Datagram_Protocol) | [CDNs](https://www.maxcdn.com/blog/anycast-ip-routing-used-maxcdn/) [(DNS)](https://en.wikipedia.org/wiki/Domain_Name_System) |

## Streaming models

| IP Multicast | IP Unicast and Client/Server Model | IP Unicast and P2P Model |
|:------------:|:----------------------------------:|:------------------------:|
| ![Multicast](data/multicast-server.svg) | ![Unicast-CS](data/unicast-server.svg) | ![Unicast-P2P](data/unicast-splitter.svg) |

| IP Multicas | IP Unicast and Client/Server Model | IP Unicast and P2P Model |
|:-----------:|:----------------------------------:|:------------------------:|
| ![Multicast](data/multicast-server.png) | ![Unicast-CS](data/unicast-server.png) | ![Unicast-P2P](data/unicast-splitter.png) |

## Lab 1: Streaming with VLC

<img src="data/VLC.svg" style="width: 300px;" align="middle"/>

#### Download a sample video

In [6]:
%%bash
if [[ ! -e Big_Buck_Bunny_small.ogv ]]; then
    wget https://upload.wikimedia.org/wikipedia/commons/7/79/Big_Buck_Bunny_small.ogv
else
    echo "File already exists :-)"
fi

File already exists :-)


#### Label the video with a duration time-stamp

In [ ]:
%%bash
if [[ ! -e Big_Buck_Bunny_small.ogv ]]; then
    ffmpeg -i Big_Buck_Bunny_small.ogv -vf "drawtext=text='%{pts\:hms}': x=(w-tw)/2: y=h-(2*lh): fontcolor=white: box=1: boxcolor=0x00000000@1" -codec:v libtheora -codec:a libvorbis LBig_Buck_Bunny_small.ogv
else
    echo "File already exists :-)"
fi

#### Run VLC serving the video over HTTP

In [1]:
%%bash --bg
cvlc LBig_Buck_Bunny_small.ogv --sout "#http{mux=ogg,dst=:8080/LBBB.ogv}" :sout-keep &
cvlc LBig_Buck_Bunny_small.ogv --sout "#duplicate{dst=http{dst=:8080/LBBB.ogv},dst=display}" :sout-keep &

[0000556332730818] dummy interface: using the dummy interface module...
[00007fee5c002dc8] core access out error: socket bind error: Permission denied
[00007fee5c002dc8] core access out error: socket bind error: Permission denied
[00007fee5c002dc8] core access out error: cannot create socket(s) for HTTP host
[00007fee5c002dc8] access_output_http access out error: cannot start HTTP server
[00007fee5c001128] stream_out_standard stream out error: no suitable sout access module for `http/ogg://:8080/BBB.ogv'
[00007fee5c000dc8] core stream output error: stream chain failed for `http{mux=ogg,dst=:8080/BBB.ogv}'
[00007fee70000c08] core input error: cannot start stream output instance, aborting
[00005563326dc6c8] core playlist: end of playlist, exiting


#### Run a (or more VLC) client(s)

In [4]:
%%bash --bg
cvlc http://localhost:8080/LBBB.ogv &
cvlc http://localhost:8080/LBBB.ogv

[000055a412cf4398] core libvlc: Running vlc with the default interface. Use 'cvlc' to use vlc without interface.
[000055b5fa613398] core libvlc: Running vlc with the default interface. Use 'cvlc' to use vlc without interface.
Failed to open VDPAU backend libvdpau_va_gl.so: cannot open shared object file: No such file or directory
Failed to open VDPAU backend libvdpau_va_gl.so: cannot open shared object file: No such file or directory


## Lab 2: Streaming with VLC and Icecast

<img src="data/icecast-1.svg" style="width: 500px;" align="middle"/>

#### Launch Icecast

In [ ]:
%%bash
icecast -b -c /etc/icecast/icecast.xml

#### Test if Icecast is running

In [ ]:
%%bash
firefox http://localhost:8000

#### Feed Icecast with a video using VLC

In [ ]:
%%bash --bg
# Run "VLC media player" and select: Media -> Stream -> Add (Select the Big_Buck_Bunny_small.ogv file) -> Stream -> Next -> Change "File" by "Icecast" -> Add (Address: "localhost", Mount Point: "test.ogg", Login:pass: "source:1qaz" -> Next -> Change "Video - H.264 + MP3 (MP4)" by "Video - Theora + Vorbis (OGG)" (Transcoding is not necessary) -> Next -> Stream.  
# Or run:  
cvlc LBig_Buck_Bunny_small.ogv --sout "#duplicate{dst=std{access=shout,mux=ogg,dst=source:hackme@localhost:8000/LBBB.ogv},dst=display}" :sout-keep

#### Running some clients

In [ ]:
%%bash --bg
cvlc http://localhost:8000/LBBB.ogv &
cvlc http://localhost:8000/LBBB.ogv

## Lab 3: Relaying

## P2PSP
* An [Application-layer Multicast](https://en.wikipedia.org/wiki/Multicast) protocol which uses a push-based (when possible) [fully-connected mesh](https://en.wikipedia.org/wiki/Network_topology) [overlay](https://en.wikipedia.org/wiki/Overlay_network) for real-time [streaming of media content](https://en.wikipedia.org/wiki/Streaming_media) between networked [entities](https://en.wikipedia.org/wiki/Process_%28computing%29).

### DBS (Data Broadcasting Set of rules)

Provides connectivity among peers using unicast infrastructure.

#### Definitions

1. ${\cal P}_i$ incomming peer.
2. $\{{\cal P}_k\} = {\cal L}_j$ list of incorporated peers (which arrived before than $P_i$). 
3. ${\cal R}$ tracker.
4. ${\cal T}_j = {\cal S}_j \cup {\cal L}_j$ $j$-th team.
4. ${\cal S}_j$ splitter of team ${\cal T}_j$.

#### Task ${\cal S}_j$.SERVE_JOINING_PEERS

1. While True:
    1. Wait for connection from ${\cal P}_i$
    2. if ${\cal P}_i \notin {\cal L}_j$:
        1. for all ${\cal P}_k \in {\cal L}_j$:
            1. $[{\cal P}_k] \Rightarrow {\cal P}_i$
        2. ${\cal L}_j$.append(${\cal P}_i$)

#### Task $P_i$.JOIN_TEAM

Run by incomming peers.

1. $[S_j] \gets R$
2. for all ${\cal P}_k\in [{\cal T}_j] \gets {\cal S}_j$
    1. $[\mathtt{hello}] \rightarrow {\cal P}_k$

#### Task $P_k$.ACCEPT_NEIGHBORS

Run by incorporated peers.

1. While True:
    1. $[\mathtt{hello}] \gets {\cal P}_i$
    2. $F[{\cal P}_k]$.insert(${\cal P}_i$) # Duplicates not allowed

#### Task ${\cal P}_k$.RECEIVE_CHUNK_AND_FLOOD

Run by peers when receiving a chunk.

1. While True:
    1. $[\mathtt{chunk\_number}, \mathtt{chunk}, \mathtt{origin}] \gets {\cal X}$
    2. if $\mathtt{buffer}[\mathtt{chunk\_number} \% \mathtt{buffer\_size}] == \mathtt{chunk\_number}$:
        1. $[\text{prune}, \mathtt{chunk\_number}] \rightarrow {\cal X}$
    3. else:
        1. $\mathtt{buffer}[\mathtt{chunk\_number}] = (\mathtt{chunk\_number}, \mathtt{chunk}, \mathtt{origin})$
        2. if ${\cal X}$ != ${\cal S}$:
            1. $D[{\cal X}] = D[{\cal X}] - 1$
            2. $F[{\cal P}_k] = F[{\cal P}_k] \cup {\cal X}$
        3. for all ${\cal P} \in F[\mathtt{origin}]$:
            1. $P[{\cal P}] = P[{\cal P}] \cup \mathtt{chunk\_number}$
        4. for all $\mathtt{chunk\_number} \in P[\mathtt{neighbor}]$:
            1. $\mathtt{buffer}[\mathtt{chunk\_number}] \rightarrow P[\mathtt{neighbor}]$
            2. $P[\mathtt{neighbor}]$.remove($\mathtt{chunk\_number}$)
            3. $D[\mathtt{neighbor}] = D[\mathtt{neighbor}] + 1$
            4. if $D[\mathtt{neighbor}] > \mathtt{MAX\_CHUNK\_DEBT}$:
                1. $D$.remove($\mathtt{neighbor}$)
                2. $F$.remove($\mathtt{neighbor}$)
        5. $\mathtt{neighbor} =$ next($P[\mathtt{neighbor}]$)

#### Task ${\cal P}_k$.CONTROL

Run by peers when receiving a control message from other peers.

1. While True:
    1. $\mathtt{message} \gets {\cal P}_j$
    2. if $\mathtt{message} == [\text{request}, \mathtt{chunk\_number}]$:
        1. $\mathtt{origin} = \mathtt{buffer}[\mathtt{chunk\_number}].\mathtt{ORIGIN}$
        2. $F[\mathtt{origin}].\text{append}({\cal P}_j)$
        3. $D[{\cal P}_j] = 0$
    3. else if $\mathtt{message} == [\text{prune}, \mathtt{chunk\_number}]$:
        1. $\mathtt{origin} = \mathtt{buffer}[\mathtt{chunk\_number}].\mathtt{ORIGIN}$
        2. $F[\mathtt{origin}].\text{remove}({\cal P}_j)$
    4. else if $\mathtt{message} == [\text{hello}]$:
        1. $F[{\cal P}_k].\text{append}({\cal P}_j)$
        2. $D[{\cal P}_j] = 0$
    5. else if $\mathtt{message} == [\text{goodbye}]$:
        1. for all $\mathtt{list} \in F$:
            1. $\mathtt{list}$.remove(${\cal P}_j$)
            2. $D$.remove(${\cal P}_j$)

## Lab 4: Scaling with P2PSP

In [2]:
!xterm